In [2]:
import tetrion.commands as cmd
import tetrion.db
import tetrion.options
import tetrion.clientcore
import datetime as dt
import tetrion.config
import statsmodels.api as sm
import pandas as pd
import zmq
import ujson
import pickle
import glob
import seaborn as sns
import numpy as np
import sys
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
symbol = pd.read_csv('/local/dist/tetrion/conf/conf/symbol.table', skiprows= 1, header = 2,  delim_whitespace=True)
from joblib import Parallel, delayed
import collections
import multiprocessing
import itertools
import redis
import json
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

/tmp/ipykernel_45291/4016521052.py:16: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


/tmp/ipykernel_45291/4016521052.py:18: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  symbol = pd.read_csv('/local/dist/tetrion/conf/conf/symbol.table', skiprows= 1, header = 2,  delim_whitespace=True)


In [3]:
import redis
import json
def get_fills_from_redis(strat = 'capital_txo_main', acc='TAIFEX100',date=dt.date.today(),night_session=False, redishost='prod1.capital.radiant-knight.com'):
    r = redis.StrictRedis(host=redishost, port=6379,db=0)
    rk = '{}:{}'.format(acc, date.strftime('%Y%m%d'))
    rk_strat = '{}:{}'.format(strat, date.strftime('%Y%m%d'))
    if night_session:
        rk = rk + 'E'
        rk_strat = rk_strat + 'E'
    if rk not in [k.decode() for k in r.keys()]:
        print('Key:{} not exists!'.format(rk))
        if rk_strat in [k.decode() for k in r.keys()]:
            rk = rk_strat
        else:
            return None
    msgs = r.lrange(rk, 0, -1)
    trd_df = pd.DataFrame([json.loads(m.decode()) for m in msgs])
    if trd_df.empty:
        print('Empty redis key')
        return None
    if trd_df['ts'][0] > 1e11:
        trd_df['time'] = trd_df['ts'].apply(lambda x:dt.datetime.fromtimestamp(x/1e6))
    else:
        trd_df['time'] = trd_df['ts'].apply(lambda x:dt.datetime.fromtimestamp(x))
    trd_df.set_index('time',inplace=True)

    return trd_df

In [5]:
def compute_fee(product, px, sz, mult):
    tax_rate = 0.00002
    fee_map = {
        'TXF': 20,
        'MXF': 12.5,
        'TMF': 8.0,
        'FXF': 20,
        'ZFF': 8.0
    }
    fix_fee = fee_map.get(product[:3], 0)
    if fix_fee == 0:
        print('Unknown product:', product)
    tax_fee = abs(px * sz * mult) * tax_rate
    return abs(sz * fix_fee) + tax_fee


def daily_pnl_neutrino(strategy, d, night=False):
    trade_fill = get_fills_from_redis(strategy, date=d, night_session=night).reset_index()
    floor = '5f' if strategy.endswith('5f') else '6f'
    
    pnl_records = []
    fill_records = []

    if trade_fill.empty:
        pnl_records.append([floor, 'NaN', 0])
    else:
        trade_fill['expiry'] = trade_fill['instr'].str[-6:]
        trade_fill['contract'] = trade_fill['instr'].str[-10:-7]
        
        for expiry, group in trade_fill.groupby('expiry'):
            group = group.sort_values(by='ts')
            pos = value = 0
            context = {'alqortake': '', 'contract': '', 'time': None}

            for _, row in group.iterrows():
                contract, px, sz, mult = row['contract'], row['px'], row['sz'], row['mult']
                pos += sz * mult
                value -= px * mult * sz
                value -= compute_fee(contract, px, sz, mult)

                if pos != 0:
                    if not context['alqortake']:
                        context.update({'alqortake': row['alqortake'], 'contract': contract})
                    if context['time'] is None:
                        context['time'] = row['time']
                else:
                    tp = 'fire' if (context['alqortake'] == 'take' and context['alqortake'] == row['alqortake'] and context['contract'] != contract) else 'quoting'
                    fill_records.append([context['time'], floor, expiry, value, tp])
                    pnl_records.append([floor, expiry, value])
                    value = 0
                    context = {'alqortake': '', 'contract': '', 'time': None}

    pnl_df = pd.DataFrame(pnl_records, columns=['floor', 'expiry', 'pnl(twd)'])
    pnl_fill = pd.DataFrame(fill_records, columns=['time', 'floor', 'expiry', 'pnl(twd)', 'type'])
    return pnl_df, pnl_fill


# 執行分析
import datetime as dt

date = dt.date(2025, 6, 17)
strategies = ['capital_neutrino_txf_5f', 'capital_neutrino_txf']

all_pnl, all_fill = [], []
for s in strategies:
    pnl_df, pnl_fill = daily_pnl_neutrino(s, date)
    all_pnl.append(pnl_df)
    all_fill.append(pnl_fill)


Key:TAIFEX100:20250617 not exists!
Key:TAIFEX100:20250617 not exists!


In [7]:

result1 = pd.concat(all_pnl).groupby(['floor', 'expiry']).sum().reset_index()
result1 = result1.pivot(index='expiry', columns='floor', values='pnl(twd)').fillna(0)
result1 = result1.loc[~(result1.sum(axis=1) == 0)]
result1.loc['sum'] = result1.sum()
result1




floor,5f,6f
expiry,,
202506,325.569,475.564
202507,1006.289,806.285
sum,1331.858,1281.849


In [9]:
# 收集所有策略的 fill 資料
result2 = pd.concat(all_fill).sort_values(by='time').reset_index(drop=True)


In [11]:
summary = (
    result2.groupby(['floor', 'type'])
    .agg(
        pnl_sum=('pnl(twd)', 'sum'),
        count=('pnl(twd)', 'count')
    )
    .unstack(level='floor')
    .fillna(0)
    .astype(int)
)




In [12]:
summary

pnl_sum       count   
floor        5f    6f    5f 6f
type                          
quoting    1331  1281     3  3

In [13]:
def compute_fee(product, px, sz, mult):
    tax_rate = 0.00002
    fee_map = {
        'TXF': 20,
        'MXF': 12.5,
        'TMF': 8.0,
        'FXF': 20,
        'ZFF': 8.0
    }
    fix_fee = fee_map.get(product[:3], 0)
    if fix_fee == 0:
        print('Unknown product:', product)
    tax_fee = abs(px * sz * mult) * tax_rate
    return abs(sz * fix_fee) + tax_fee


def daily_pnl_neutrino(strategy, d, night=False):
    trade_fill = get_fills_from_redis(strategy, date=d, night_session=night).reset_index()
    floor = '6f'  # 只處理六樓
    
    pnl_records = []
    fill_records = []

    if trade_fill.empty:
        pnl_records.append([floor, 'NaN', 0])
    else:
        trade_fill['expiry'] = trade_fill['instr'].str[-6:]
        trade_fill['contract'] = trade_fill['instr'].str[-10:-7]
        
        for expiry, group in trade_fill.groupby('expiry'):
            group = group.sort_values(by='ts')
            pos = value = 0
            context = {'alqortake': '', 'contract': '', 'time': None}

            for _, row in group.iterrows():
                contract, px, sz, mult = row['contract'], row['px'], row['sz'], row['mult']
                pos += sz * mult
                value -= px * mult * sz
                value -= compute_fee(contract, px, sz, mult)

                if pos != 0:
                    if not context['alqortake']:
                        context.update({'alqortake': row['alqortake'], 'contract': contract})
                    if context['time'] is None:
                        context['time'] = row['time']
                else:
                    tp = 'fire' if (context['alqortake'] == 'take' and context['alqortake'] == row['alqortake'] and context['contract'] != contract) else 'quoting'
                    fill_records.append([context['time'], floor, expiry, value, tp])
                    pnl_records.append([floor, expiry, value])
                    value = 0
                    context = {'alqortake': '', 'contract': '', 'time': None}

    pnl_df = pd.DataFrame(pnl_records, columns=['floor', 'expiry', 'pnl(twd)'])
    pnl_fill = pd.DataFrame(fill_records, columns=['time', 'floor', 'expiry', 'pnl(twd)', 'type'])
    return pnl_df, pnl_fill


In [14]:
import datetime as dt

date = dt.date(2025, 6, 17)
strategy = 'capital_neutrino_txf'  # 六樓

# 執行日盤
pnl_day, fill_day = daily_pnl_neutrino(strategy, date, night=False)

# 執行夜盤
pnl_night, fill_night = daily_pnl_neutrino(strategy, date, night=True)

# 合併結果
result1 = pd.concat([pnl_day, pnl_night]).groupby(['floor', 'expiry']).sum().reset_index()
result1 = result1.pivot(index='expiry', columns='floor', values='pnl(twd)').fillna(0)
result1 = result1.loc[~(result1.sum(axis=1) == 0)]
result1.loc['sum'] = result1.sum()

result2 = pd.concat([fill_day, fill_night]).sort_values(by='time').reset_index(drop=True)

# 輸出
with pd.option_context('display.float_format', '{:,.0f}'.format):
    print(result1)


Key:TAIFEX100:20250617 not exists!
Key:TAIFEX100:20250617E not exists!
floor     6f
expiry      
202506   429
202507 1,580
sum    2,009
